## **IU000135 PERSONLIZATION AND MACHINE LEARNING**
##### Mini-Project: Exploring Recommender Systems for Course Selection by Yifan Feng
*For detailed information, please read technical report*

## Reference

#### Main Source Code
* [IU000135 Week 1: Content-based Recom System](https://git.arts.ac.uk/lmccallum/personalisation-21-22/blob/master/Week%201.2%20-%20Similarity%20Based%20Filtering%20.ipynb)
* [Course Recommendation System Project](https://github.com/IntellectualCoders/Course-Recommendation-System)
* [Movie Collabrative Filtering Project](https://github.com/KelvinLam05/item-based_collaborative_filtering/blob/main/Item-based_collaborative_filtering.ipynb)
* [Web App for Course Recommendation](https://github.com/SagarBapodara/Coursera-Course-Recommendation-System)

#### Dataset
* [HavardX: EdX Harvard&MIT](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/26147)
* [Kaggle: Merged MOOCs courses (Udacity+Coursea)](https://www.kaggle.com/datasets/ayushbatra/online-mooc)
* [Kaggle: EdX](https://www.kaggle.com/datasets/khusheekapoor/edx-courses-dataset-2021)

#### Academic Papers
* [A Collaborative Recommendation System for Online Courses Recommendations](https://ieeexplore.ieee.org/abstract/document/8876926)
* [Application of Collaborative Filtering Recommendation Algorithm in Internet Online Courses](https://dl.acm.org/doi/10.1145/3469968.3469992)
* [Recommendation System for Elective Courses using Content-based Filtering and Weighted Cosine Similarity](https://ieeexplore.ieee.org/document/9702788)
* [Course recommendation based on semantic similarity analysis](https://ieeexplore.ieee.org/document/8088011)

## Content-based Approach

#### Set-up Env

In [ ]:
!pip install rake_nltk 

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity 
from sklearn.feature_extraction.text import CountVectorizer

In [67]:
#Text Processing Setup
from rake_nltk import Rake #https://pypi.org/project/rake-nltk/ 
import nltk
nltk.download('punkt')
nltk.download("wordnet")
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /Users/ivanfeng/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ivanfeng/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/ivanfeng/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


True

In [55]:
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

#### Reorganize Database

* Important metadata for course recommendation system includes: course name, course description, course objective, course difficulty
* Drop course ratings for two reasons:
    1. not provided in Edx data source 
    2. ratings are unevenly distributed across different classes
* Replace old headers with new catagories

In [3]:
dataset_path = './online_course/'
df_mooc = pd.read_csv (dataset_path+'MOOC.csv', encoding='cp1252') #Encode invalid error, solution on https://stackoverflow.com/questions/46000191/utf-8-codec-cant-decode-byte-0x92-in-position-18-invalid-start-byte
df_edx = pd.read_csv (dataset_path+'EdX.csv')

In [4]:
df_mooc.isnull().sum() #check zero value 
#df_edx.isnull().sum()

Course Name                            0
University / Industry Partner Name     0
Difficulty Level                       4
Course Rating                          0
Course URL                             0
Course Description                     8
all_skill                             15
dtype: int64

In [5]:
df_mooc_rename = df_mooc.rename(columns={'University / Industry Partner Name': 'University', 
                                         'Difficulty Level': 'Course Difficulty',
                                         'all_skill':'Course Objective',}) 
df_edx_rename = df_edx.rename(columns={'Name': 'Course Name', 
                                       'Difficulty Level': 'Course Difficulty',
                                       'Link':'Course URL',
                                      'About':'Course Description',
                                      'Course Description': 'Course Objective'}) 

In [6]:
del df_mooc_rename['Course Rating'] #drop rating column 

In [7]:
df_mooc_rename.head(5)

,Course Name,University,Course Difficulty,Course URL,Course Description,Course Objective
0,Write A Feature Length Screenplay For Film Or ...,Michigan State University,Beginner,https://www.coursera.org/learn/write-a-feature...,Write a Full Length Feature Film Script\n\nIn ...,Drama Comedy peering screenwriting film D...
1,Business Strategy: Business Model Canvas Analy...,Coursera Project Network,Beginner,https://www.coursera.org/learn/canvas-analysis...,"By the end of this guided project, you will be...",Finance business plan persona (user experien...
2,Silicon Thin Film Solar Cells,École Polytechnique,Advanced,https://www.coursera.org/learn/silicon-thin-fi...,This course consists of a general presentation...,chemistry physics Solar Energy film lambda...
3,Finance for Managers,IESE Business School,Intermediate,https://www.coursera.org/learn/operational-fin...,"When it comes to numbers, there is always more...",accounts receivable dupont analysis analysis...
4,Retrieve Data using Single-Table SQL Queries,Coursera Project Network,Beginner,https://www.coursera.org/learn/single-table-sq...,In this course you’ll learn how to effectively...,Data Analysis select (sql) database manageme...


In [8]:
df_edx_rename.head(5)

,Course Name,University,Course Difficulty,Course URL,Course Description,Course Objective
0,How to Learn Online,edX,Beginner,https://www.edx.org/course/how-to-learn-online,Learn essential strategies for successful onli...,"Designed for those who are new to elearning, t..."
1,Programming for Everybody (Getting Started wit...,The University of Michigan,Beginner,https://www.edx.org/course/programming-for-eve...,"This course is a ""no prerequisite"" introductio...",This course aims to teach everyone the basics ...
2,CS50's Introduction to Computer Science,Harvard University,Beginner,https://www.edx.org/course/cs50s-introduction-...,An introduction to the intellectual enterprise...,"This is CS50x , Harvard University's introduct..."
3,The Analytics Edge,Massachusetts Institute of Technology,Intermediate,https://www.edx.org/course/the-analytics-edge,"Through inspiring examples and stories, discov...","In the last decade, the amount of data availab..."
4,Marketing Analytics: Marketing Measurement Str...,"University of California, Berkeley",Beginner,https://www.edx.org/course/marketing-analytics...,This course is part of a MicroMasters® Program,Begin your journey in a new career in marketin...


In [9]:
#Combine two dataset 
# Method 1: df_mooc_rename.append(df_edx_rename)
# Method 2: 
df_all_course = pd.concat([df_mooc_rename, df_edx_rename], axis=0, join='outer') 
# df_all_course_save = df_all_course.to_csv(dataset_path+'mooc_update.csv') 
df_new_data = df_all_course [['Course Name', 'University','Course Difficulty','Course Description','Course Objective']]

In [10]:
df_new_data.head(5)

,Course Name,University,Course Difficulty,Course Description,Course Objective
0,Write A Feature Length Screenplay For Film Or ...,Michigan State University,Beginner,Write a Full Length Feature Film Script\n\nIn ...,Drama Comedy peering screenwriting film D...
1,Business Strategy: Business Model Canvas Analy...,Coursera Project Network,Beginner,"By the end of this guided project, you will be...",Finance business plan persona (user experien...
2,Silicon Thin Film Solar Cells,École Polytechnique,Advanced,This course consists of a general presentation...,chemistry physics Solar Energy film lambda...
3,Finance for Managers,IESE Business School,Intermediate,"When it comes to numbers, there is always more...",accounts receivable dupont analysis analysis...
4,Retrieve Data using Single-Table SQL Queries,Coursera Project Network,Beginner,In this course you’ll learn how to effectively...,Data Analysis select (sql) database manageme...


In [11]:
df_new_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4505 entries, 0 to 719
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Course Name         4505 non-null   object
 1   University          4505 non-null   object
 2   Course Difficulty   4501 non-null   object
 3   Course Description  4497 non-null   object
 4   Course Objective    4490 non-null   object
dtypes: object(5)
memory usage: 211.2+ KB


#### Data Processing
* For content-based approach, first check value in chosen columns. Replace NaN with empty string for Text Process
* Use nltk package [Rake function](https://pypi.org/project/rake-nltk/) to extract keywords (based on relevance ranking) from both columns and save them in new columns 

In [11]:
display(df_new_data['Course Description'].isnull().values.any(), 
        df_new_data['Course Objective'].isnull().values.any())

True

True

In [13]:
df_new_data = df_new_data.copy()
df_new_data['Course Description'] = df_new_data['Course Description'].fillna('').apply(str) #To resolve string Error 

df_new_data['bag_of_words_description']= '' #create a new column
for index, row in df_new_data.iterrows():
    description = row['Course Description']

    #use Rake() to extract keywords
    r = Rake()
    r.extract_keywords_from_text(description)
    key_words_dict_scores = r.get_word_degrees() 
    row['bag_of_words_description'] = list(key_words_dict_scores.keys())

In [256]:
#del df_new_data['bag_of_words']

In [14]:
df_new_data['Course Objective'] = df_new_data['Course Objective'].fillna('').apply(str)
df_new_data['bag_of_words_objective']= ''
for index, row in df_new_data.iterrows():
    description = row['Course Objective']
    
    #use Rake() to extract keywords, https://pypi.org/project/rake-nltk/
    r = Rake()
    r.extract_keywords_from_text(description)
    key_words_dict_scores = r.get_word_degrees() 
    row['bag_of_words_objective'] = list(key_words_dict_scores.keys())

In [15]:
df_new_data['Course Difficulty'] = df_new_data['Course Difficulty'].str.lower()
df_new_data.head(5)

,Course Name,University,Course Difficulty,Course Description,Course Objective,bag_of_words_description,bag_of_words_objective
0,Write A Feature Length Screenplay For Film Or ...,Michigan State University,beginner,Write a Full Length Feature Film Script\n\nIn ...,Drama Comedy peering screenwriting film D...,"[write, full, length, feature, film, script, n...","[drama, comedy, peering, screenwriting, film, ..."
1,Business Strategy: Business Model Canvas Analy...,Coursera Project Network,beginner,"By the end of this guided project, you will be...",Finance business plan persona (user experien...,"[end, guided, project, fluent, identifying, cr...","[finance, business, plan, persona, user, exper..."
2,Silicon Thin Film Solar Cells,École Polytechnique,advanced,This course consists of a general presentation...,chemistry physics Solar Energy film lambda...,"[course, consists, general, presentation, sola...","[chemistry, physics, solar, energy, film, lamb..."
3,Finance for Managers,IESE Business School,intermediate,"When it comes to numbers, there is always more...",accounts receivable dupont analysis analysis...,"[comes, numbers, always, meets, eye, operation...","[accounts, receivable, dupont, analysis, accou..."
4,Retrieve Data using Single-Table SQL Queries,Coursera Project Network,beginner,In this course you’ll learn how to effectively...,Data Analysis select (sql) database manageme...,"[course, ’, learn, effectively, retrieve, data...","[data, analysis, select, sql, database, manage..."


#### Model Construction: Vectorization and Cosine Similarity
* Create a matirx list based on word frequency 
* Apply stemming and lemmatization to clean data. See [IU000131 NLP Week 2.1](https://git.arts.ac.uk/lmccallum/nlp-21-22/blob/master/NLP%20Week%202.1.ipynb)
* Caculate cosine similarity. See [IU000135 Personlization & ML Week 1.2](https://git.arts.ac.uk/lmccallum/personalisation-21-22/blob/master/Week%201.2%20-%20Spotify%20Playlists%20task.ipynb)

In [16]:
df_new_data = df_new_data.copy()
df_new_data['bag_of_words_description'] = df_new_data['bag_of_words_description'].apply(lambda x: ' '.join(x)) #Convert from list to string
df_new_data['bag_of_words_objective'] = df_new_data['bag_of_words_objective'].apply(lambda x: ' '.join(x))

In [86]:
df_new_data['tags'] = df_new_data['Course Difficulty'] + ' ' + df_new_data['bag_of_words_description'] + df_new_data['bag_of_words_objective']

In [87]:
df_new_data.loc[0, 'tags']

0    beginner write full length feature film script...
0    beginner learn essential strategies successful...
Name: tags, dtype: object

In [ ]:
#Stemming and Lemmatization Method 1 

"""
def stemming (text):
    stemmed_list = []
    for tag in text.split():
        stemmed_list.append(PorterStemmer().stem(tag))
        
    return " ".join(stemmed_list)

def lemmatizing (text):
    lemmed_list = []
    for i in text.split():
        lemmed_list.append(WordNetLemmatizer().lemmatize(i))
    return " ".join(lemmed_list)
        
def textprocessing(text):
    return lemmatizing(stemming(text))
    
"""

In [106]:
#Stemming and Lemmatization Method 2 
df_new_data['cleaned_tags'] = df_new_data['tags'].astype(str).apply(PorterStemmer().stem)
df_new_data['cleaned_tags'] = df_new_data['cleaned_tags'].astype(str).apply(WordNetLemmatizer().lemmatize) 

In [112]:
count_matrix = CountVectorizer().fit_transform(df_new_data['cleaned_tags'].fillna(' ')).toarray()
similarity_matrix = cosine_similarity(count_matrix)
similarity_matrix

array([[1.        , 0.08812699, 0.0631446 , ..., 0.05129892, 0.06668179,
        0.07392961],
       [0.08812699, 1.        , 0.03844732, ..., 0.03346581, 0.02900074,
        0.06251954],
       [0.0631446 , 0.03844732, 1.        , ..., 0.0527535 , 0.08000131,
        0.02956562],
       ...,
       [0.05129892, 0.03346581, 0.0527535 , ..., 1.        , 0.03979186,
        0.03431318],
       [0.06668179, 0.02900074, 0.08000131, ..., 0.03979186, 1.        ,
        0.01486753],
       [0.07392961, 0.06251954, 0.02956562, ..., 0.03431318, 0.01486753,
        1.        ]])

#### Explore Results
* Define a recommendation function in the steps:
    1. Access course title from index
    2. Access each row based on given title and gather all similarity matrix 
    3. Sort similarity and discard the first element (aka input, distances=0)
    4. Use index to find course title, print top 5 results

In [135]:
df_new_data.rename(columns = {'Course Name':'course_name'}, inplace = True)

def recommend_top5(course):
    #access index from title 
    course_index = df_new_data[df_new_data['course_name'] == course].index[0]
    
    #access the row on given course title to find all similarity matrix
    distances = similarity_matrix[course_index]
    
    #sort similarity values in decending order, discard the first element as it is the input (similarity=1) 
    course_list = sorted(list(enumerate(distances)),reverse=True, key=lambda x:x[1])[1:6]
    
    #use index to locate course title 
    for i in course_list:
        print(df_new_data.iloc[i[0]].course_name)

In [144]:
df_new_data['course_name'][300:310]

300    Android App Components - Services, Local IPC, ...
301                          VR and 360 Video Production
302               RPA Lifecycle: Development and Testing
303                          Methods of Surface Analysis
304                          Startups in open innovation
305                               Python Data Structures
306    Blockchain, Cryptoassets, and Decentralized Fi...
307                         Data Processing Using Python
308    International Law in Action: the Arbitration o...
309                        Applied Text Mining in Python
Name: course_name, dtype: object

In [143]:
recommend_top5("App Design and Development for iOS") 

Build Your Own iOS App
iOS App Development Basics
Best Practices for iOS User Interface Design
Build a Firebase Android Application
Toward the Future of iOS Development with Swift


In [141]:
recommend_top5("Big data and Language 2")

Big data and Language 1
Natural Language Processing
Bioconductor for Genomic Data Science
Model Building and Validation
Applied Machine Learning in Python


In [145]:
recommend_top5("Applied Text Mining in Python")

Applied Social Network Analysis in Python
Introduction to Data Science in Python
Applied Machine Learning in Python
Applied Plotting, Charting & Data Representation in Python
Clinical Natural Language Processing


## Collaborative-filtering Approach 

* **Limitation:** input data structure does not meet user-user filtering approach. Therefore this experiment focuses on item-item method.

#### Loading Database

In [323]:
dataset_path = './online_course/'
df_harvard_edx = pd.read_csv(dataset_path +'harvard_edx.csv')

In [324]:
df_harvard_edx.head(3)

,course_id,userid_DI,registered,viewed,explored,certified,final_cc_cname_DI,LoE_DI,YoB,gender,grade,start_time_DI,last_event_DI,nevents,ndays_act,nplay_video,nchapters,nforum_posts,roles,incomplete_flag
0,HarvardX/CB22x/2013_Spring,MHxPC130442623,1,0,0,0,United States,NaN,NaN,NaN,0,12/19/12,11/17/13,NaN,9.0,NaN,NaN,0,NaN,1.0
1,HarvardX/CB22x/2013_Spring,MHxPC130275857,1,0,0,0,United States,NaN,NaN,NaN,0,2/8/13,11/17/13,NaN,16.0,NaN,NaN,0,NaN,1.0
2,HarvardX/CB22x/2013_Spring,MHxPC130539455,1,1,0,0,France,NaN,NaN,NaN,0,1/1/13,5/14/13,42.0,6.0,NaN,3.0,0,NaN,NaN


In [325]:
df_harvard_edx.rename(columns = {'userid_DI':'student_id'}, inplace = True)

In [294]:
"""
df_harvard_edx['student_id'].nunique()
df_harvard_edx['course_id'].describe()
df_harvard_edx['viewed'].nunique()
"""

"\ndf_harvard_edx['userid_DI'].nunique()\ndf_harvard_edx['course_id'].describe()\ndf_harvard_edx['viewed'].nunique()\n"

In [326]:
#Check enrollment number of chosen module
df_harvard_edx[df_harvard_edx['course_id']== 'HarvardX/CB22x/2013_Spring']

,course_id,student_id,registered,viewed,explored,certified,final_cc_cname_DI,LoE_DI,YoB,gender,grade,start_time_DI,last_event_DI,nevents,ndays_act,nplay_video,nchapters,nforum_posts,roles,incomplete_flag
0,HarvardX/CB22x/2013_Spring,MHxPC130442623,1,0,0,0,United States,NaN,NaN,NaN,0,12/19/12,11/17/13,NaN,9.0,NaN,NaN,0,NaN,1.0
1,HarvardX/CB22x/2013_Spring,MHxPC130275857,1,0,0,0,United States,NaN,NaN,NaN,0,2/8/13,11/17/13,NaN,16.0,NaN,NaN,0,NaN,1.0
2,HarvardX/CB22x/2013_Spring,MHxPC130539455,1,1,0,0,France,NaN,NaN,NaN,0,1/1/13,5/14/13,42.0,6.0,NaN,3.0,0,NaN,NaN
3,HarvardX/CB22x/2013_Spring,MHxPC130088379,1,1,0,0,United States,NaN,NaN,NaN,0,2/18/13,3/17/13,70.0,3.0,NaN,3.0,0,NaN,NaN
4,HarvardX/CB22x/2013_Spring,MHxPC130024894,1,1,0,0,United States,NaN,NaN,NaN,0.07,1/24/13,8/3/13,175.0,9.0,NaN,7.0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29997,HarvardX/CB22x/2013_Spring,MHxPC130069603,1,1,0,0,Other East Asia,NaN,NaN,f,0,8/27/13,8/27/13,20.0,1.0,NaN,1.0,0,NaN,NaN
29998,HarvardX/CB22x/2013_Spring,MHxPC130142150,1,0,0,0,Other East Asia,Bachelor's,1990.0,m,0,8/27/13,8/27/13,1.0,1.0,NaN,NaN,0,NaN,NaN
29999,HarvardX/CB22x/2013_Spring,MHxPC130239562,1,0,0,0,Unknown/Other,Secondary,1990.0,m,0,8/27/13,11/17/13,NaN,NaN,NaN,NaN,0,NaN,NaN
30000,HarvardX/CB22x/2013_Spring,MHxPC130388295,1,0,0,0,United States,Master's,1986.0,m,0,9/5/13,11/17/13,NaN,NaN,NaN,NaN,0,NaN,NaN


#### Model Construction: item-item perspective

* Reorganized database by course_id (5). Caculate registration number and completion number for each class
* Create a course matrix based on course completion. Use course_id as similarity index and student_id as unique catagories.
    1. fill in zero for NaN value for similarity ranking

In [327]:
df_harvard_edx.groupby('course_id').agg(registration_num = ('student_id', 'nunique'),  
                                view_number = ('viewed', 'sum')).sort_values(by = 'registration_num', ascending = False).head()

,registration_num,view_number
course_id,,
HarvardX/CS50x/2012,169621,106086
HarvardX/ER22x/2013_Spring,57406,32161
HarvardX/PH207x/2012_Fall,41592,24279
HarvardX/PH278x/2013_Spring,39602,15016
HarvardX/CB22x/2013_Spring,30002,16314


In [328]:
df_course_matrix = df_harvard_edx.pivot_table(index = 'course_id', columns = ['student_id'], values = 'viewed').fillna(0)

In [329]:
df_course_matrix.head()

student_id,MHxPC130000002,MHxPC130000004,MHxPC130000006,MHxPC130000007,MHxPC130000011,MHxPC130000013,MHxPC130000015,MHxPC130000016,MHxPC130000017,MHxPC130000021,...,MHxPC130597650,MHxPC130597651,MHxPC130597653,MHxPC130597654,MHxPC130597659,MHxPC130597661,MHxPC130597665,MHxPC130597667,MHxPC130597670,MHxPC130597674
course_id,,,,,,,,,,,,,,,,,,,,,
HarvardX/CB22x/2013_Spring,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HarvardX/CS50x/2012,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0
HarvardX/ER22x/2013_Spring,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
HarvardX/PH207x/2012_Fall,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
HarvardX/PH278x/2013_Spring,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Explore Results
* Use [corrwith()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corrwith.html) to identify the Pearson correlation coefficient for each student with each other (check shared course)
* Drop NaN values and re-ranking based on correlation 
* Instead of recommending courses, this approach can base on shared interest in class to suggest top 10 potential peers to the enquiring student

**[Caculate Similarity Methods](https://towardsdatascience.com/calculate-similarity-the-most-relevant-metrics-in-a-nutshell-9a43564f533e)**

In [330]:
def get_student_recommendations(df, student_id):
    
    recommendations = df.corrwith(df[student_id], method='kendall')
    recommendations.dropna(inplace = True)
    recommendations = pd.DataFrame(recommendations, columns = ['correlation']).reset_index()
    recommendations = recommendations.nlargest(10, columns = ['correlation'], keep='first') 

    return recommendations 

In [331]:
result = get_student_recommendations(df_course_matrix, 'MHxPC130597661')

In [332]:
display(result)

,student_id,correlation
1,MHxPC130000006,1.0
7,MHxPC130000026,1.0
29,MHxPC130000109,1.0
31,MHxPC130000114,1.0
39,MHxPC130000137,1.0
42,MHxPC130000140,1.0
43,MHxPC130000141,1.0
51,MHxPC130000168,1.0
58,MHxPC130000188,1.0
64,MHxPC130000207,1.0
